# Pinecone Basics

Una delle difficoltà maggiori, legata al mondo degli LLM e della Generative AI e alle grandissime moli di dati richieste per il training e a volte per operare, è l'**efficient data processing**.
Molte delle applicazioni di AI più rcenti utilizzano i **vector embeddings**: chatbots, question-answering systems, machine translation, etc.
I vector embeddings sono fondamentali per aquisire **understanding** e **long term memory**.
Convertire in vector embeddings le descrizioni dei libri contenuti in un dataset di 5000 unità (per realizzare un sistema di raccomandazione basato su similarità), produce un file di circa 200MB, si immagini una intera biblioteca. Gestire file di tali dimensioni mediante un semplice **csv** diventa estremamente inefficiente.
A tal fine tornano utili i **Vector Database** come per esempio **Pinecone, Chroma, Milvus e Qdrant**.
**Pinecone** è stato realizzato per la gestione di **high dimensional vectors** e mette a disposizione una potente **ricerca semantica** tra tali vettori, integrandosi perfettamente con **OpenAI**.
I Vector Database, a differenda dei SQL database, sono stati pensati per fare **storing e querying** di dati **unstructured** (testo, immagini, audio). La ricerca nei Vector Database avviene per similarità combinando diversi algoritmi ottimizati al fine di individuare il cosiddetto **Approximate Nearest Neighbor(ANN)**.


## Inizializzazione di Pinecone

In [16]:
# Carico le API Keys
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(filename="pinecone.env"), override=True)

True

In [17]:
#per evitare il warning della cella seguente
from tqdm.autonotebook import tqdm 

In [18]:
import pinecone 

# inizializzazione pinecone - sto usando utenza ros.moscato@gmail.com
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

In [19]:
# Verifico la versione
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.4')

## Pinecone Indexes

In [20]:
# Listo gli indici esistenti
pinecone.list_indexes()

['langchain-pinecone']

### Creo un index

In [21]:
# creo un indice - configurazione servizio gratuito
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ....')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')

Index langchain-pinecone already exists!


In [22]:
# Verifico un indice
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

### Cancello un index

In [18]:
# cancello un indice
index_name = input('Enter Pinecone index to delete: ')
if index_name in pinecone.list_indexes():
    print(f'Deleting index {index_name} ... ')
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Deleting index langchain-pinecone ... 
Done


### Statistiche index

In [23]:
# Recupero statistiche indice
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Namespaces
Pinecone ci permette di partizionare i vettori presenti in un indice in **namespaces**.
Le query e le altre operazioni saranno quindi limitate a un namespace, per cui richieste differenti agiranno su diversi subsets dell'indice. I namespaces sono unici e identificati da un nome univoco (namespace name), che di default è una empty string.

### Inserimento di vettori in un index

In [25]:
# inserimento di alcuni(5) vettori randomici nel Pinecone Index
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde') #abbiamo 5 vettori e quindi 5 elementi per gli IDs
ids

['a', 'b', 'c', 'd', 'e']

In [27]:
# Nota: si usa 'upsert' e non insert. Upsert è una operazione singola per fare insert o update.
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [28]:
# Recupero statistiche indice
#index_name = 'langchain-pinecone'
#index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 5e-05,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

### Operazione di update

In [29]:
# update di un vettore
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

In [30]:
# Recupero statistiche indice
#index_name = 'langchain-pinecone'
#index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 5e-05,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

### Fetching di un vettore

In [31]:
# fetching
index = pinecone.Index('langchain-pinecone')
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'metadata': {},
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

### Cancellazione vettori

In [34]:
# Cancellazione vettori
index.delete(ids=['b', 'c'])

{}

In [38]:
# Adesso i vettori sono solo 3
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [36]:
# La ricerca di un vettore che non esiste non produce un errore ma un vettore vuoto {}
index.fetch(ids=['b'])

{'namespace': '', 'vectors': {}}

In [42]:
# Cancello TUTTI i vettori
# index.delete(delete_all=True)

In [14]:
index.delete(ids=['a'])

{}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

### Query

In [49]:
# Creo nuovamente alcuni(5) vettori randomici nel Pinecone Index (perchè li ho cancellati prima)
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde') #abbiamo 5 vettori e quindi 5 elementi per gli IDs

In [50]:
# Inserisco i 5 vettori randomici
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [70]:
# Creo 2 vettori randomici per poi effettuare una query
queries = [[random.random() for _ in range(1536)] for v in range(1)]

In [71]:
len(queries)

1

In [72]:
# In questo caso la query restituisce i 3 vettori più simili (top_k=3)
# La prima tripletta è costituita dai matches per il primo vettore (queries)
# La seconda tripletta è costituita dai matches per il secondo vettore (queries)
index.query(
    vector=queries,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'd', 'score': 0.747123718, 'values': []},
             {'id': 'e', 'score': 0.744924605, 'values': []},
             {'id': 'b', 'score': 0.743837237, 'values': []}],
 'namespace': ''}